In [1]:
# Chase Abram
using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random

# Load data
df = DataFrame()
df = CSV.read("psetOne.csv", DataFrame)

In [104]:
# names(df)

# Get market t = 17
df17 = df[df."Market" .== 17, :]

# Store variables for market 17
X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)
p_given = df17."Price"
df17

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,17,1,0.920128,3.0,1,0,0,0.0276024,0.0282533
2,17,1,1.97453,5.0,1,0,0,0.219283,0.296879
3,17,1,1.94548,3.0,1,1,0,0.150052,0.0497719
4,17,1,1.86088,5.0,1,1,0,0.387576,0.301157
5,17,1,1.98307,6.5,0,0,1,0.0714796,0.033719
6,17,1,3.8985,10.0,1,0,1,0.0523355,0.601297
7,17,1,2.86479,10.0,0,0,1,0.152532,0.0601887


In [59]:
# function u_bar_jt(delta, sigma, X, zeta)
#     return delta .+ X*sigma*zeta
# end

# function Prij(u_bar)
#     return exp.(u_bar)./sum(exp.(u_bar))
# end

# Why is this jumping to 0? (prices are going negative and massive)

# Shares
function s_logit(p, alpha, beta, X, xi)
    
    # Compute numerators for each s_j
    nums = exp.(X*beta - alpha.*p + xi)
    
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)

    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    return Om
end

# Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* -s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) - Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    # Initialize
    jacomp = zeros(length(p), length(p))
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Hessian of shares
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    # Ownership
    oms = Omega_star(br)
    
    # Fill in
    for i in 1:length(p)
        for k in 1:length(p)
#             jacomp[i,k] = sum([sh[i,j,k]*p[j]*oms[i,j] for j in 1:length(p)]) + sj[i,k]*oms[i,k]
            jacomp[i,k] = -(sh[i,:,k]' * (oms[i,:] .* p))[1] #+ oms[i,k] * sj[i,k]
        end
    end
    return jacomp
end

# Jacobian of fixed point expression
function J_fp_logit(p, alpha, beta, X, xi, br)
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # (Jacobian of Omega)*p
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
    # Omega
    om = Omega(p, alpha, beta, X, xi, br)
#     println("sj: ", sj)
#     println("jacomp: ", jacomp)
#     println("om: ", om)
    return sj - jacomp - om
end

# Solve for fixed point
function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 1000)
    # Initialize price and update storage
    p = p0
    pnew = zeros(length(p0))
    
    # Initialize Jacobian
    J = zeros(length(p0), length(p0))
    
    # Iterator
    it = 0
    
    # Difference in updating
    diff = Inf
    println("p0: ", p0)
    
    # Continue until converged or maximum iterations
    while diff > tol && it < maxiter
        println("p: ", p)
        
        # Jacobian of FP expression
        J .= J_fp_logit(p, alpha, beta, X, xi, br)
        println("J: ", J)
#         println("J inv: ", inv(J))
#         println("J under fp: ", J \ fp_logit(p, alpha, beta, X, xi, br))
#         println("size(p): ", size(p))
#         println("size(J under fp): ", size(J \ fp_logit(x, alpha, beta, X, xi, br)))
        
        # Update
        pnew = (p - (J \ fp_logit(p, alpha, beta, X, xi, br)))[:,1]
        it += 1

        println("pnew: ", pnew)
        
        # Get difference
        diff = maximum(abs.(pnew - p))
        println("diff: ", diff)
        
        p = pnew
    end
    
    println("Took ", it, " iterations")
    return p
end



# Squared distance of FP expression from origin
# function fp_logit_2(p, alpha, beta, X, xi, br)
#     return (fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br))[1]
# end


# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     sp = s_logit(p, alpha, beta, X, xi)
#     Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
#     s_jac = s_logit_jac(p, alpha, beta, X, xi)
#     jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
#     jac = 2*(sp + Om*p)' * (s_jac + jacomp + Om)
#     for i in 1:length(p)
#         storage[i] = jac[i]
#     end
# #     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
# #     storage = 0
# #     return storage
    
# end

# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     for k in 1:length(p)
#         storage[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
#     println("stor: ", storage)
# #     return storage
# end

# function fp_logit_2_jac(p, alpha, beta, X, xi, br)
#     s = s_logit(p, alpha, beta, X, xi)
#     oms = Omega_star(br)
#     sj = s_logit_jac(p, alpha, beta, X, xi)
#     sh = s_logit_hess(p, alpha, beta, X, xi)
    
#     out = zeros(length(p))
#     for k in 1:length(p)
#         out[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
#     end
# #     println("stor: ", storage)
#     return out
# end

# function basic_newton_zero(f, g, init, tol = 1e-14, maxiter = 10000)
#     x = init
#     x_new = init
#     diff = Inf
#     it = 1
# #     println("x init: ", init)
#     while diff > tol && it < maxiter
# #         println("it: ", it)
# #         println("g(x): ", g(x))
#         x_new -= 0.5.*g(x)
# #         println("x_new: ", x_new)
#         diff = maximum(abs.(x_new - x))
# #         println("diff: ", diff)
#         it += 1
#         x = x_new
#     end
#     return x
# end


# function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
#     f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    
# #     g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
#     g!(x) = fp_logit_2_jac(x, alpha, beta, X, xi, br)
    
#     out = basic_newton_zero(f, g!, p)
# #     Optim.Options(f_tol = tol)
# #     Optim.Options(time_limit = 10.0)
# #     opt = optimize(f, g!, p, LBFGS(), Optim.Options(time_limit = 5.0))
# #     opt = optimize(f, x -> ForwardDiff.gradient(f, p), p, BFGS(), Optim.Options(time_limit = 10.0))
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end



solve_fp_logit (generic function with 3 methods)

In [72]:
p = ones(size(X,1),1) .* 1.0
# p .= p_given
# p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'


# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1)) .* 0.0
# println("xi: ", xi)

# println("p: ", p)
# println("delta: ", X*beta - alpha .* p + xi)


println("shares: ", s_logit(p, alpha, beta, X, xi))
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)
# println("shares_jac: ", s_logit_jac(p, alpha, beta, X, xi))

s_logit_hess(p, alpha, beta, X, xi)

Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)
# println("fp_logit: ", fp_logit(p, alpha, beta, X, xi, Br))
fp_logit(p, alpha, beta, X, xi, Br)

# J_fp_logit(p, alpha, beta, X, xi, Br)

# out = solve_fp_logit(p, alpha, beta, X, xi, Br)
# println("out: ", out)
# println("fp_logit: ", fp_logit(out, alpha, beta, X, xi, Br))



# fp_logit_2(p, alpha, beta, X, xi, Br)
# st = zeros(length(p))
# fp_logit_2_jac(st, p, alpha, beta, X, xi, Br)
# println(st)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


# op = solve_fp_logit(alpha, beta, X, xi, Br)
# println("op: ", op)
# println("opam: ", opam)
# println("test on fp: ", fp_logit(opam, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

# Gradient must be wrong.
# Check whole construction and parameter choice.


shares: [0.00029341911289887894; 0.0021680902857082822; 0.00010794285928225993; 0.0007975958427155957; 0.0035745765708455634; 0.8746700481466118; 0.11837371870450338]


7×1 Array{Float64,2}:
 -0.0005846714640853634
 -0.0043201702475706635
 -0.00021559247925446672
 -0.001593024923718797
  0.0035383125997220477
  0.8657965469808248
  0.11717282091093119

In [41]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]







In [42]:
function f1(x, y, z) 
    return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
end

function g1(storage, x, y, z)
    storage[1] = 2*(x[1] - 5 - y)
    storage[2] = 2*(x[2] - 3 - z)
end

function g2(x,y,z)
    out = zeros(2)
    out[1] = 2*(x[1] - 5 - y)
    out[2] = 2*(x[2] - 3 - z)
    
    return out
end

function tester()
    a = 1000
    b = -1000
    f(x) = f1(x,a,b)
#     g!(storage, x) = g1(storage, x, a, b)
    g!(x) = g2(x,a,b)
    
    out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
#     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
#     return opt, opt.minimizer, opt.minimum
    return out
end

tester()



2-element Array{Float64,1}:
 1005.0
 -997.0

In [70]:
p = ones(size(X,1),1) .* 1.0
# p[1] = 50
# p[2] = 30
# p[3] = 90
# p[4] = -12
# p[5] = -40
# p[6] = 0
# p[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
# println(s_logit_jac(p, alpha, beta, X, xi))
ForwardDiff.jacobian(s, p)

# ForwardDiff.gradient(fp, p)


# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
J_ex = J_fp_logit(p, alpha, beta, X, xi, Br)
println(J_ex)
# println(maximum(abs.(ForwardDiff.jacobian(fp, p) .- J_ex)))
println(ForwardDiff.jacobian(fp, p))
# println(inv(J_ex))

[0.0008727277308654773 -7.605723129685889e-6 -1.8863166935850017e-7 -1.3938099869248953e-6 -6.246622984529345e-6 -0.0015284982482105759 -0.00020685974334824474; -7.605723129685891e-6 0.006400040770708964 -1.3938099869248953e-6 -1.0298940184637844e-5 -4.6156647661556936e-5 -0.011294159303145172 -0.001528498248210576; -1.89518735976878e-7 -1.4003645719315781e-6 0.00032280922990471746 -1.0317339910446653e-6 -2.308810856478474e-6 -0.0005649473897043272 -7.645731499942015e-5; -1.4003645719315784e-6 -1.0347372380957437e-5 -1.0317339910446657e-6 0.0023786637026706006 -1.7059932940339566e-5 -0.004174427955469709 -0.0005649473897043273; 1.252234578345595e-5 9.252831548416369e-5 4.606713568973342e-6 3.403926499284905e-5 -0.02126277969578708 0.018569136311739884 0.0025130593222085864; 0.0030641169862345484 0.022640932304973393 0.0011272256445798898 0.00832913352395407 0.018569136311739877 -0.6776819407849809 0.6149253414458312; 0.00041468314020216867 0.0030641169862345484 0.00015255340188079274 0

In [84]:
nI = 20
nJ = 3
nN = 5

delta_1 = zeros(3)
X_1 = zeros(3,nN)
sigma_1 = 0.0 .* I(size(X,2))
zeta_1 = zeros(nN, nI)

delta_2 = zeros(3)
delta_2[1] = 40
delta_2[2] = 20
delta_2[3] = 20
X_2 = zeros(nJ,nN)
sigma_2 = 0.0 .* I(size(X,2)).* 0.1
zeta_2 = zeros(nN, nI)

sigma_3 = 0.1 .* I(nN)
delta_3 = zeros(nJ)
X_3a = zeros(nJ,nN)

rng = MersenneTwister(1234)
zeta_3 = randn(rng, nN, nI)

rng = MersenneTwister(1234)
X_3b = randn(rng, size(X_3a))
X_3c = X_3b .* 10
X_3d = abs.(X_3c)
    

# function u_bar(delta, X, sigma, zeta)
    
#     u_bar = zeros(size(zeta,2), size(delta,1))
    
#     for i in 1:size(u_bar, 1)
#         for j in 1:size(u_bar, 2)
#             u_bar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
#         end
#     end
    
#     return u_bar
# end

# function pr(u_bar)
    
#     p = zeros(size(u_bar,1), size(u_bar,2))
    
#     for i in 1:size(p,1)
#         for j in 1:size(p,2)
#             p[i,j] = exp(u_bar[i,j])/(1 + sum(exp.(u_bar[i,:])))
#         end
#     end
    
#     return p
# end

function pr(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    ubar = zeros(T, size(zeta,2), size(delta,1))
    
    for i in 1:size(ubar, 1)
        for j in 1:size(ubar, 2)
            ubar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
        end
    end
    
    p = zeros(T, size(ubar,1), size(ubar,2))
    
    for i in 1:size(p,1)
        for j in 1:size(p,2)
            p[i,j] = exp(ubar[i,j])/(1 + sum(exp.(ubar[i,:])))
        end
    end
    
    return p
end




function sHat(delta::AbstractVector{T}, X, sigma, zeta) where T
#     ubar = zeros(size())
#     ubar = u_bar(delta, X, sigma, zeta)
#     p = pr(ubar)
    
    p = pr(delta, X, sigma, zeta)
    
    s = zeros(T, size(delta))
    
    for j in 1:length(s)
        s[j] = 1/size(zeta,2) * sum(p[:,j])
    end
    
    return s
end

s1 = sHat(delta_1, X_1, sigma_1, zeta_1)
s2 = sHat(delta_2, X_2, sigma_2, zeta_2)
s3a = sHat(delta_3, X_3a, sigma_3, zeta_3)
s3b = sHat(delta_3, X_3b, sigma_3, zeta_3)
s3c = sHat(delta_3, X_3c, sigma_3, zeta_3)
s3d = sHat(delta_3, X_3d, sigma_3, zeta_3)

println("s1: ", s1)
println("s2: ", s2)
println("s3a: ", s3a)
println("s3b: ", s3b)
println("s3c: ", s3c)
println("s3d: ", s3d)

s1: [0.25, 0.25, 0.25]
s2: [0.999999995877693, 2.0611536139418496e-9, 2.0611536139418496e-9]
s3a: [0.25, 0.25, 0.25]
s3b: [0.25396627272710803, 0.23546917816783988, 0.2598787383914339]
s3c: [0.302761733713778, 0.18340110893056388, 0.3631032237834327]
s3d: [0.20718104155891687, 0.1964933641693537, 0.23876547586390232]


In [92]:
function sHat_jac(delta, X, sigma, zeta)
    p = pr(delta, X, sigma, zeta)
    
    s = sHat(delta, X, sigma, zeta)
    
    sj = zeros(size(delta,1), size(delta,1))
    
    for j in 1:size(sj,1)
        for k in 1:size(sj,2)
            if j == k
                sj[j,k] = 1/size(zeta,2) * sum(p[:,j] .* (1 .- p[:,j]))
            else
                sj[j,k] = 1/size(zeta,2) * sum(-p[:, j] .* p[:, k])
            end
        end
    end
    
    return sj
end

function sHat_log_jac(delta, X, sigma, zeta)
    s = sHat(delta, X, sigma, zeta)
    sj = sHat_jac(delta, X, sigma, zeta)
    
    slj = zeros(size(delta,1), size(delta,1))
    
    for j in 1:size(slj, 1)
        for k in 1:size(slj,2)
            slj[j,k] = sj[j,k]/s[k]
        end
    end
    
    return slj
end

s1_jac = sHat_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_jac: ", s1_jac)
s1_log_jac = sHat_log_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_log_jac: ", s1_log_jac)

s1_jac: [0.1875 -0.0625 -0.0625; -0.0625 0.1875 -0.0625; -0.0625 -0.0625 0.1875]
s1_log_jac: [0.75 -0.25 -0.25; -0.25 0.75 -0.25; -0.25 -0.25 0.75]


In [100]:
function sHat_inv(s, X, sigma, zeta, tol = 1e-14, maxiter = 1e5)
    delta = zeros(size(X,1))
    diff = maximum(log.(s) - log.(sHat(delta, X, sigma, zeta)))
    it = 0
    while diff > tol && it < maxiter
#         println("it: ", it)
        
        shat = sHat(delta, X, sigma, zeta)
#         println("shat: ", shat)
        
#         if diff > 1e-3
#             println("Contraction")
            inc = log.(s) - log.(shat)
#         else
#             println("Newton")
            
#             slj = sHat_log_jac(delta, X, sigma, zeta)
# #             println("slj: ", slj)
#             inc = -slj \ log.(shat)
#         end
        
        delta += inc
        
        diff = maximum(abs.(inc))
#         println("diff: ", diff)
        it += 1
    end
    println("Loop exited")
    
    println("it: ", it)
    println("diff: ", diff)
    
    
    return delta
end

println("s1_inv: ", sHat_inv(s1, X_1, sigma_1, zeta_1))
println("s2_inv: ", sHat_inv(s2, X_2, sigma_2, zeta_2))
# println("s3a_inv: ", sHat_inv(s3a, X_3a, sigma_3, zeta_3))
# println("s3b_inv: ", sHat_inv(s3b, X_3b, sigma_3, zeta_3))
# println("s3c_inv: ", sHat_inv(s3c, X_3c, sigma_3, zeta_3))
# println("s3d_inv: ", sHat_inv(s3d, X_3d, sigma_3, zeta_3))

Loop exited
it: 0
diff: 0.0
s1_inv: [0.0, 0.0, 0.0]
Loop exited
it: 100000
diff: 9.999750006399497e-6
s2_inv: [11.5129554604045, -8.487044539595498, -8.487044539595498]


In [75]:
delta_1 = ones(3)
delta_2 = 2
delta_3 = 40
X_1 = 10 .* randn(3,nN)
sigma_1 = 0.1 .* I(size(X,2))
zeta_1 = randn(nN, 10000)

shl(x) = log.(sHat(x, X_1, sigma_1, zeta_1))
println(sHat_log_jac(delta_1, X_1, sigma_1, zeta_1))
ForwardDiff.jacobian(shl, delta_1)

[0.4695771366223323 -0.10315171193658614 -0.2709627240753898; -0.13164998854187257 0.28770562927898313 -0.17255100985230817; -0.24182392814892698 -0.12065986678121084 0.517404227219085]


3×3 Array{Float64,2}:
  0.469577  -0.13165   -0.241824
 -0.103152   0.287706  -0.12066
 -0.270963  -0.172551   0.517404

In [ ]:
X
X'